In [ ]:
#data transfer between unrelated databases (from trino to oracle in this example)
import trino
import easygui
import pandas as pd
import cx_Oracle
from datetime import datetime
import time
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings("ignore") 

#variables
start_time = datetime.now()
today_date = start_time.strftime("%Y-%m-%d__%H_%M_%S")
global_path = os.getcwd() + '\\' #directory with all the files: LOAD_LOG, BAD_LOG, extract_query
print('Start: ' + start_time.strftime("%Y-%m-%d %H:%M:%S"))



In [ ]:
#connecting to target_base:

pass_target_base = easygui.passwordbox("target_base password: ")

#variables below to fill in
USERNAME = ''
HOST = ''
PORT = ''
SID = ''

con_target_base = cx_Oracle.connect(f'{USERNAME}/{pass_target_base}@{HOST}:{PORT}/{SID}')
cursor_target_base = con_target_base.cursor()

#number of rows before data insert
cursor_target_base.execute('SELECT COUNT(*) FROM target_table')
pre_rows_nbr = cursor_target_base.fetchone()[0]

#headers download for bad_log
df_target_base = pd.read_sql("select * from target_table where rownum = 0", con_target_base)
columns_df_target_base = df_target_base.columns.to_list()

#max date from target table, to load only increment from extract table
cursor_target_base.execute('SELECT max(date_field) FROM target_table')
max_date_target_table = cursor_target_base.fetchone()[0]



In [ ]:
#connecting to extract_base

login_extract_base = easygui.enterbox("extract_base login: ")
password_extract_base = easygui.passwordbox("extract_base password: ")

con_extract_base = trino.dbapi.connect(host='', port='', http_scheme='', auth=trino.auth.BasicAuthentication(login_extract_base, password_extract_base))#, verify=False/True)
cursor_extract_base = con_extract_base.cursor()



In [ ]:
#downloading data from extract_base to variable data_to_transfer
print('Start downloading data from extract_base')
extract_base_sql_path = global_path + 'extract_query.sql' #this file contains extract data query
with open(extract_base_sql_path,'r') as file:
    sel_extract_base = file.read()

sel_extract_base = sel_extract_base.format("'"+str(max_date_target_table)+"'") #max data from target table before data insert

cursor_extract_base.execute(sel_extract_base)
data_to_transfer = cursor_extract_base.fetchall()

rows_number_extract_base = len(data_to_transfer)
print(str(rows_number_extract_base) + ' rows were downloaded from extract_base')



In [ ]:
#insert data into target_base
print('Starting data insert to target_base')
bad_list = []
cnt = 0

list_of_fields = ':1'
for i in range(2,len(data_to_transfer[0])+1):
    list_of_fields = list_of_fields + ',:' + str(i)

for row in tqdm(data_to_transfer, desc = 'Progress', unit = 'iter'):
    try:
        cursor_target_base.execute('INSERT INTO target_table VALUES ({})'.format(list_of_fields), row)
    except:
        bad_list.append(row)
    
    cnt += 1
    if cnt%1000 == 0:
        con_target_base.commit()

#before the first insertion of data, the table in target_database must be created. It must have the same columns and data types as the query result from extract_base
cursor_target_base.execute('SELECT COUNT(*) FROM target_table')
post_rows_bnr = cursor_target_base.fetchone()[0]
print('Data inserted into target_table')



In [ ]:
con_target_base.commit()
con_target_base.close()
con_extract_base.close()



In [ ]:
#creation of logs
print('Logs are being created')
end_time = datetime.now()
execution_time = round((end_time - start_time).total_seconds() / 60,0)

log_text = "At the day " + \
    str(start_time.strftime('%Y-%m-%d %H:%M:%S')) + \
    " from extract_base have been extracted " + \
    str(rows_number_extract_base) + \
    " rows. Into target_table have been inerted " + \
    str(post_rows_bnr - pre_rows_nbr) +\
    ' rows in ' +\
    str(execution_time) +\
    ' minutes. To BAD file ' +\
    str(len(bad_list)) +\
    ' rows were inserted.\n'

load_log_path = global_path + 'LOAD_LOG.txt'
with open(load_log_path,'a') as file:
    file.write(log_text)


#bad log
bad_log_path = global_path + 'BAD_LOG_'+ str(today_date) + '.txt'
bad_list.insert(0, columns_df_target_base) #header with column names
if len(bad_list) > 1:
    for bad in bad_list:
        with open(bad_log_path,'a') as file:
            file.write(str(bad)+'\n')

print('End. Check logs.')
print('Window will close on 5 seconds')

time.sleep(5)